
# Benchmark Validation

Select a benchmark scenario via `SELECTED_SCENARIO` to compare measured TTFT/ITL values with the analytical estimators.


In [ ]:

from dataclasses import dataclass
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from utils.config import get_model_config, get_hardware_config
from utils.math_utils import (
    prefill_compute_time,
    prefill_memory_time,
    decode_compute_time,
    decode_memory_time,
)

pd.options.display.max_rows = 20
pd.options.display.max_columns = 20


@dataclass(frozen=True)
class EvalModelBenchmarkConfiguration:
    data_path: Path
    model_name: str
    csv_hardware_label: str
    hardware_key: str
    running_tokens_cap: float
    prefill_running_tokens_cap: float


In [ ]:

EVAL_MODEL_BENCHMARK_CONFIGURATION = {
    "llama33_70b_fp8_tp2": EvalModelBenchmarkConfiguration(
        data_path=Path("tested_benchmarks/nim_llama33_70b_v1.8.0_2xH100_fp8TP2.csv"),
        model_name="llama33_70B",
        csv_hardware_label="H100_80G",
        hardware_key="H100_80GB_FP8_TP2",
        running_tokens_cap=3.03e5,
        prefill_running_tokens_cap=6.25e4
    ),
    "llama31_8b_fp8_tp1": EvalModelBenchmarkConfiguration(
        data_path=Path("tested_benchmarks/nim_llama3.1_8b_v1.8.0_1xH100_80GB.csv"),
        model_name="llama31_8B",
        csv_hardware_label="H100_80G",
        hardware_key="H100_80GB_FP8_TP1",
        running_tokens_cap=7.5e5,
        prefill_running_tokens_cap= 8e4
    ),
}

SELECTED_SCENARIO = "llama33_70b_fp8_tp2"
CONFIG = EVAL_MODEL_BENCHMARK_CONFIGURATION[SELECTED_SCENARIO]
print(f"Scenario: {SELECTED_SCENARIO}")
print(f"Running tokens cap: {CONFIG.running_tokens_cap:,.0f}")


## Load configuration and raw measurements

In [ ]:

df_raw = pd.read_csv(CONFIG.data_path)

hardware_values = df_raw["hardware"].unique()
if len(hardware_values) != 1 or hardware_values[0] != CONFIG.csv_hardware_label:
    raise ValueError(
        f"Unexpected hardware labels {hardware_values}. Expected '{CONFIG.csv_hardware_label}'."
    )

df_raw["hardware_key"] = CONFIG.hardware_key
model_cfg = get_model_config(CONFIG.model_name)
hardware_cfg = get_hardware_config(CONFIG.hardware_key)

print(f"Loaded rows: {len(df_raw)}")
display(df_raw.head())



## Compute analytical estimates

For each scenario we compute the prefill (TTFT) and decode (ITL) bottlenecks, keep both compute/memory components, and convert them to milliseconds.


In [ ]:
phi_term_pow = 1
def estimate_row(row):
    S = float(row["concurrency"])
    L_prompt = float(row["input_tokens"])
    L_decode = L_prompt + float(row["output_tokens"])

    prefill_compute = prefill_compute_time(S, L_prompt, model_cfg, hardware_cfg, running_tokens_cap=CONFIG.prefill_running_tokens_cap, prefill_mult_cost=1, phi_term_pow=phi_term_pow )
    prefill_memory = prefill_memory_time(S, L_prompt, model_cfg, hardware_cfg)
    decode_compute = decode_compute_time(S, L_decode, model_cfg, hardware_cfg)
    decode_memory = decode_memory_time(S, L_decode, model_cfg, hardware_cfg, CONFIG.running_tokens_cap)

    ttft_compute_ms = float(prefill_compute * 1e3)
    ttft_memory_ms = float(prefill_memory * 1e3)
    itl_compute_ms = float(decode_compute * 1e3)
    itl_memory_ms = float(decode_memory * 1e3)

    running_tokens = S * L_decode

    return pd.Series({
        "prefill_compute_ms": ttft_compute_ms,
        "prefill_memory_ms": ttft_memory_ms,
        "ttft_model_ms": max(ttft_compute_ms, ttft_memory_ms),
        "prefill_limit": "compute" if ttft_compute_ms >= ttft_memory_ms else "memory",
        "decode_compute_ms": itl_compute_ms,
        "decode_memory_ms": itl_memory_ms,
        "itl_model_ms": max(itl_compute_ms, itl_memory_ms),
        "decode_limit": "compute" if itl_compute_ms >= itl_memory_ms else "memory",
        "running_tokens": min(running_tokens, CONFIG.running_tokens_cap),
        # "running_tokens_utilization": running_tokens / CONFIG.running_tokens_cap,
    })


df_eval = df_raw.join(df_raw.apply(estimate_row, axis=1))
df_eval["ttft_ratio"] = df_eval["ttft_model_ms"] / df_eval["TTFT_ms"]
df_eval["itl_ratio"] = df_eval["itl_model_ms"] / df_eval["ITL_ms"]
df_eval["ttft_delta_ms"] = df_eval["ttft_model_ms"] - df_eval["TTFT_ms"]
df_eval["itl_delta_ms"] = df_eval["itl_model_ms"] - df_eval["ITL_ms"]

df_eval = df_eval.sort_values(["input_tokens", "concurrency"]).reset_index(drop=True)
display(df_eval[[
    "input_tokens", "concurrency", "running_tokens",
    "TTFT_ms", "ttft_model_ms", "ttft_delta_ms",
    "ITL_ms", "itl_model_ms", "itl_delta_ms", "prefill_limit", "decode_limit"
]])


In [ ]:
# save results 
eval_folder = "evaluations_results"
df_eval.to_csv(f"{eval_folder}/eval_benchmark-{SELECTED_SCENARIO}.csv")

## Error summary by prompt length

In [ ]:

summary = (
    df_eval
    .groupby("input_tokens")[[
        "TTFT_ms", "ttft_model_ms", "ttft_ratio",
        "ITL_ms", "itl_model_ms", "itl_ratio"
    ]]
    .agg(["mean", "median"])
)
summary



## Plots

The helpers below make it easy to compare measured vs. modelled values for any prompt length.


In [ ]:

def plot_metric_vs_concurrency(metric_name, observed_col, model_col, prompt_tokens):
    subset = df_eval[df_eval["input_tokens"] == prompt_tokens].sort_values("concurrency")
    if subset.empty:
        raise ValueError(f"No rows with input_tokens={prompt_tokens}")
    melted = subset.melt(
        id_vars=["concurrency"],
        value_vars=[observed_col, model_col],
        var_name="series",
        value_name="value",
    )

    title = f"{metric_name} vs concurrency (prompt={prompt_tokens})"
    fig = px.line(melted, x="concurrency", y="value", color="series", markers=True, title=title)
    fig.update_layout(xaxis_title="Concurrency", yaxis_title=f"{metric_name} [ms]")
    fig.show()


for prompt_tokens in sorted(df_eval["input_tokens"].unique()):
    plot_metric_vs_concurrency("TTFT_ms", "TTFT_ms", "ttft_model_ms", prompt_tokens)

for prompt_tokens in sorted(df_eval["input_tokens"].unique()):
    plot_metric_vs_concurrency("ITL_ms", "ITL_ms", "itl_model_ms", prompt_tokens)


In [ ]:

context_batches = sorted(df_eval['concurrency'].unique())


def plot_metric_vs_context(metric_name, observed_col, model_col, concurrency):
    subset = df_eval[df_eval['concurrency'] == concurrency].sort_values('input_tokens')
    if subset.empty:
        raise ValueError(f'No rows with concurrency={concurrency}')
    melted = subset.melt(
        id_vars=['input_tokens'],
        value_vars=[observed_col, model_col],
        var_name='series',
        value_name='value',
    )
    title = f"{metric_name} vs context length (concurrency={concurrency})"
    fig = px.line(
        melted,
        x='input_tokens',
        y='value',
        color='series',
        markers=True,
        title=title,
    )
    fig.update_layout(xaxis_title='Prompt tokens', yaxis_title=f'{metric_name} [ms]')
    fig.show()


for concurrency in context_batches:
    plot_metric_vs_context('TTFT_ms', 'TTFT_ms', 'ttft_model_ms', concurrency)

for concurrency in context_batches:
    plot_metric_vs_context('ITL_ms', 'ITL_ms', 'itl_model_ms', concurrency)


In [ ]:

fig_ttft = px.scatter(
    df_eval,
    x="TTFT_ms",
    y="ttft_model_ms",
    color="input_tokens",
    hover_data=["concurrency", "prefill_limit", "running_tokens_utilization"],
    title="TTFT: measured vs modelled",
)
fig_ttft.add_trace(
    go.Scatter(
        x=[df_eval["TTFT_ms"].min(), df_eval["TTFT_ms"].max()],
        y=[df_eval["TTFT_ms"].min(), df_eval["TTFT_ms"].max()],
        mode="lines",
        name="ideal",
    )
)
fig_ttft.update_layout(xaxis_title="Measured TTFT [ms]", yaxis_title="Model TTFT [ms]")
fig_ttft.show()

fig_itl = px.scatter(
    df_eval,
    x="ITL_ms",
    y="itl_model_ms",
    color="input_tokens",
    hover_data=["concurrency", "decode_limit", "running_tokens_utilization"],
    title="ITL: measured vs modelled",
)
fig_itl.add_trace(
    go.Scatter(
        x=[df_eval["ITL_ms"].min(), df_eval["ITL_ms"].max()],
        y=[df_eval["ITL_ms"].min(), df_eval["ITL_ms"].max()],
        mode="lines",
        name="ideal",
    )
)
fig_itl.update_layout(xaxis_title="Measured ITL [ms]", yaxis_title="Model ITL [ms]")
fig_itl.show()



---

Adjust `SELECTED_SCENARIO` or extend `EVAL_MODEL_BENCHMARK_CONFIGURATION` as additional benchmark files become available.
